In [1]:
#download modis satelite image by date 
#python3.9 ./download_modis.py
from pyhdf.SD import SD, SDC
import numpy as np
import matplotlib.pyplot as plt
#from ipynb.fs.full.utils import *
import glob
from glob import glob
import os
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import re
import csv
import h5py
import numpy as np
import json
import datetime
from datetime import timedelta
#import pywt
diameter = 5

# NDVI: MOD13Q1

!pwd
!ls -altr
!


/home/fun/plant-predictor
total 66952
-rw-rw-r--.  1 fun fun    35149 Jun 25 15:55 LICENSE
-rw-rw-r--.  1 fun fun      141 Jun 25 15:59 README.txt
-rw-rw-r--.  1 fun fun   222350 Jun 27 12:52 2d_linear_layer_example.png
-rw-rw-r--.  1 fun fun    88307 Jun 27 13:18 3d_linear_layer_example.png
-rw-rw-r--.  1 fun fun     1873 Jun 27 13:20 linear layer tutorial.ipynb
-rw-rw-r--.  1 fun fun     7862 Jun 27 14:47 patch_embed_attention.ipynb
-rw-rw-r--.  1 fun fun  2223493 Jul  1 19:43 download_imerg.ipynb
-rw-rw-r--.  1 fun fun  1090420 Jul  2 11:01 CA_wildfires.json
-rw-rw-r--.  1 fun fun  1005307 Jul  2 11:02 file_creator.ipynb
-rw-rw-r--.  1 fun fun 63470545 Jul  2 15:00 ca_soil_color.tif
drwxrwxr-x.  2 fun fun      198 Jul  2 16:01 Soil_CA
drwxrwxr-x.  8 fun fun      153 Jul  4 15:58 STTran
-rw-rw-r--.  1 fun fun     3087 Jul  4 18:39 sttran_class.py
drwx------. 41 fun fun     4096 Jul  5 20:23 ..
-rw-rw-r--.  1 fun fun    39043 Jul  5 20:24 download_modis_ndvi.ipynb
drwxrwxr-x.  8 fun f

In [2]:
def download_file(folder, filename, datestr):
     
    username_file = open("/home/fun/profile/modis_username.txt", "r")
    password_file = open("/home/fun/profile/modis_password.txt", "r")
    username = username_file.readline()
    password = password_file.readline()
    
    url = generate_modis_url(datestr)
    url = url + filename
    if len(filename) == 0:
        filename = 'fires_index_' + datestr + '.html'
    
    print ('downloading file ', url)
    
    r = requests.get(url, auth = (username, password))
    if r.status_code == 200:
        print ('writing to', folder + filename)
        with open(folder + filename, 'wb') as out:
            for bits in r.iter_content():
                out.write(bits)
    else:
        print ('download error ', r.status_code)
        
def parse_html(html_file):
    '''
    parse html to get file list
    '''

    with open(html_file, 'r') as input:
        soup = BeautifulSoup(input, "html.parser").find_all(lambda t: t.name == "a" and t.text.startswith('MOD') and t.text.endswith('hdf'))
        filelist = []
        for it in soup:
            filelist.append(it["href"])
        return filelist

def generate_modis_url(datestr):
    '''
    compose url using date  'YYYY.MM.DD'
    '''
    url = 'https://e4ftl01.cr.usgs.gov/MOLT/MOD13Q1.006/'+ datestr[0:4] + '.' + datestr[5:7] + '.'+ datestr[8:10] + '/'
    
    return url
    

In [3]:
def download_one_fire(folder):
    download_html(folder)
    download_hdf(folder)

def download_html(folder):
    profile = folder + 'profile.json'
    f = open(profile)
    info = json.load(f)
    
    start = datetime.datetime.strptime(info['start'], '%Y-%m-%d')
    end = datetime.datetime.strptime(info['end'], '%Y-%m-%d')
    begin_date = start - timedelta(days=16)
    final_date = end + timedelta(days=16)
    
    it_date = begin_date
    while (it_date < final_date):
        datestr = it_date.strftime('%Y-%m-%d')
        print(datestr)
        url = generate_modis_url(datestr)
        download_file(folder, "", datestr)
        try:
            filelist = parse_html(folder + 'fires_index_' + datestr + '.html')
            it_date = it_date + timedelta(days=16)
        except FileNotFoundError:
            it_date = it_date + timedelta(days=1)
            pass
        
def download_hdf(folder):
    globdirc = folder + '*.html'
    for filename in glob(globdirc):
        datestr = filename[-15:-5]
        filelist = parse_html(filename)
        for hdf in filelist:
            if ('h08v04' in hdf) or ('h08v05' in hdf):
                print('downloading ', hdf, 'to ', folder)
                download_file(folder, hdf, datestr)

In [ ]:
download_one_fire('/home/fun/wildfire_data/August_Complex_includes_Doe_Fire_2020-08-16/')

In [ ]:
folders = glob('/home/fun/wildfire_data/*')
for folder in folders:
    download_one_fire(folder)

In [ ]:
hdf_0 = SD('/home/fun/wildfire_data/August_Complex_includes_Doe_Fire_2020-08-16/MOD13Q1.A2020225.h08v05.006.2020241233111.hdf')
hdf = SD('/home/fun/wildfire_data/August_Complex_includes_Doe_Fire_2020-08-16/MOD13Q1.A2020337.h08v05.006.2020358165155.hdf')


hdf_dict = hdf.datasets()
for idx,sds in enumerate(hdf_dict.keys()):
    print ('idx:', idx, '    sds:', sds)
  
ndvi = hdf.select(0).get()
ndvi_0 = hdf_0.select(0).get()


plt.imshow(ndvi)

In [5]:
h08v04_info = dict([
    ('ll_lon', -131.0149),
    ('ll_lat', 39.7081),
    ('ul_lon', -156.8405),
    ('ul_lat', 49.8983),
    ('ur_lon', -140.2398),
    ('ur_lat', 50.1258),
    ('lr_lon', -117.2848),
    ('lr_lat', 39.8699),
    ('lat_delta', 10),
    ('lat_min', 40),
    ('lat_max', 50),
])

h08v05_info = dict([           
    ('ll_lon', -115.3696),
    ('ll_lat', 29.8308),
    ('ul_lon', -130.5407),
    ('ul_lat', 40.0000),
    ('ur_lon', -117.3606),
    ('ur_lat', 40.0852),
    ('lr_lon', -103.6998),
    ('lr_lat', 29.9063),
    ('lat_delta', 10),
    ('lat_min', 30),
    ('lat_max', 40),
])

def get_coords(lat, lon, MOD):
    lon_index = 0
    lat_index = 0
    if (h08v05_info['lat_min'] < lat <= h08v05_info['lat_max']): 
        MOD_lat = MOD + '_h08v05_lat'
        MOD_lon = MOD + '_h08v05_lon'
    else:
        MOD_lat = MOD + '_h08v04_lat'
        MOD_lon = MOD + '_h08v04_lon'
    while coords[MOD_lat][lat_index][0] > lat:
        lat_index += 1
    while coords[MOD_lon][lat_index][lon_index] < lon:
        lon_index += 1
    # print(coords[MOD_lat][lat_index][lon_index], coords[MOD_lon][lat_index][lon_index])
    return lat_index, lon_index

def get_subimg(lat, lon, radius, hdf_link, MOD):
    lat_index, lon_index = get_coords(lat, lon, MOD)
    hdf = SD(hdf_link)
    ndvi = hdf.select(0).get()
    # print(lat_index, lon_index)
    subimg = ndvi[lat_index-radius:lat_index+radius, lon_index-radius:lon_index+radius]
    return subimg

def load_lon_lat(txt):
    arr = np.loadtxt(txt, str)
    arr = np.char.replace(arr, ',', '')
    arr = arr.astype(float)
    length = int((arr.shape[0]) ** 0.5)
    arr = np.reshape(arr, (length, length))
    
    return arr

In [6]:
coords = dict()
txts = glob('/home/fun/wildfire_coords/*.txt')
for txt in txts:
    name = txt[-22:-4]
    arr = load_lon_lat(txt)
    coords[name] = arr

In [8]:
def write_imgs(folder, radius):
    profile = folder + '/profile.json'
    with open(profile) as f:
        prof = json.load(f)
        lat = prof['info']['latitude']
        lon = prof['info']['longitude']
    if (h08v05_info['lat_min'] < lat <= h08v05_info['lat_max']): 
        tile = '*h08v05*'
    else:
        tile = '*h08v04*'
    mod_link = folder + '/' + tile
    mods = glob(mod_link)
    for mod in mods: 
        date = datetime.datetime(int(mod[-36:-32]), 1, 1) + datetime.timedelta(days = int(mod[-32:-29])-1)
        subimg = get_subimg(lat, lon, radius, mod, mod[-45:-38])
        file = folder + '/' + mod[-45:-38] + '_' + date.strftime('%Y-%m-%d') + '.npy'
        np.save(file, subimg)
        img = plt.imshow(subimg)
        plt.show()

In [ ]:
radius = 100
write_imgs('/home/fun/wildfire_data/Saddle_Ridge_Fire_2019-10-10', radius)

In [ ]:
type(ndvi)
print(ndvi.size)
print(ndvi.shape)

ndvi0_sub = ndvi_0[1000:1039,2000:2039]
ndvi_sub = ndvi[1000:1039,2000:2039]
print(150*40)

plt.figure()

#subplot(r,c) provide the no. of rows and columns
f, axarr = plt.subplots(2,1) 

# use the created array to output your multiple images. In this case I have stacked 4 images vertically
axarr[0].imshow(ndvi0_sub)
axarr[1].imshow(ndvi_sub)


In [ ]:
import math
Pixels  = 4800
VERTICAL_TILES = 18
HORIZONTAL_TILES = 36
# EARTH_RADIUS = 6371007.181
# EARTH_WIDTH = 2 * math.pi * EARTH_RADIUS


#TILE_WIDTH = EARTH_WIDTH / HORIZONTAL_TILES
TILE_WIDTH = 1200000
TILE_HEIGHT = TILE_WIDTH
Pixel_SIZE = TILE_WIDTH / Pixels

print('TILE_WIDTH', ':', TILE_WIDTH)
print('TILE_HEIGHT', ':', TILE_HEIGHT)
print('Pixel_SIZE', ':', Pixel_SIZE)

#converting from geographic coordinates
def lat_lon_to_modis(lat, lon):
    
    return int(h), int(v)

In [ ]:
import random

#candidates for presentation:
#1)  midsize file = '/home/fun/data/AL162003/10_modis_satellite_20031005.npy'
#2) /home/fun/data/AL052001/10_modis_satellite_20010828.npy

file ='/home/fun/data/AL052001/10_modis_satellite_20010828.npy'

rgb = np.load(file)

print (rgb.shape)


plt.imshow(rgb, interpolation='nearest', aspect='auto')
#plt.imshow(T20)

#plt.show()

rgb.shape

red = rgb[:, :, 0]
green = rgb[:, :, 1]
blue = rgb[:, :, 2]


def neighborRGB(color):
    for row in range(len(color)):

        found_swath = False
        swath_start = 0
        len_swath = 0
        idx = 0

        curr_row = color[row]
        
        while idx < len(curr_row):
            
            if found_swath == True:
                if curr_row[idx][0] != -2.8672 or curr_row[idx][1] != -2.8672 or curr_row[idx][2] != -2.8672:
                    found_swath = False
                else:
                    len_swath += 1
            elif curr_row[idx][0] == -2.8672  and curr_row[idx][1] == -2.8672 and curr_row[idx][2] == -2.8672:
                found_swath = True
                len_swath = 1
                swath_start = idx
            idx += 1

        existing_vals = []

        left_length = max(1,int(len_swath/2))
        right_length = max(1,int(len_swath - left_length))
        swath_end = swath_start + len_swath + 1
        #print("left:",left_length, "right:", right_length, "swath start:", swath_start)
        
        extra_right = max(0, swath_end+right_length - len(curr_row))
        left_length += extra_right
        
        extra_left = max(0, left_length - swath_start)
        right_length += extra_left

        existing_vals = np.append(existing_vals, curr_row[swath_start-left_length:swath_start])
        existing_vals = np.append(existing_vals, curr_row[swath_end:swath_end+right_length])
        
        
        for new_row_idx in (idx+1, idx+10):
            if new_row_idx >= 0 and new_row_idx < len(color):  # add previous row
                new_row = color[new_row_idx]
                for i in range(swath_start-left_length, swath_start):
                    if i >=0 and i < len(new_row):
                        if new_row[i][0] != -2.8672 or new_row[i][1] != -2.8672 or new_row[i][2] != -2.8672:
                            existing_vals = np.append(existing_vals, new_row[i])
                            
                for i in range(swath_end, swath_end+right_length+1):
                    if i >=0 and i < len(new_row):
                        if new_row[i][0] != -2.8672 or new_row[i][1] != -2.8672 or new_row[i][2] != -2.8672:
                            existing_vals = np.append(existing_vals, new_row[i])
        
    
      
    
        for idx in range(len_swath):
            rand_idx = random.randint(0, len(existing_vals)-1)
            curr_row[swath_start+idx] = existing_vals[rand_idx]
            

#neighborRGB(rgb)
    


rgb_new = np.dstack([red, green, blue])

fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(rgb_new, interpolation='nearest')
plt.show()


fig.tight_layout()
plt.show()


R_coeffs2 = pywt.dwt2(red, 'bior1.3')
G_coeffs2 = pywt.dwt2(green, 'bior1.3')
B_coeffs2 = pywt.dwt2(blue, 'bior1.3')

titles = ['Approximation', ' Horizontal detail',
          'Vertical detail', 'Diagonal detail']

R_LL, (R_LH, R_HL, R_HH) = R_coeffs2
G_LL, (G_LH, G_HL, G_HH) = G_coeffs2
B_LL, (B_LH, B_HL, B_HH) = B_coeffs2


def get_circluar_mask(arrlen, cx, cy, r1, r2):
    x = np.arange(0, arrlen)
    y = np.arange(0, arrlen)
    arr = np.zeros((y.size, x.size))
    # The two lines below could be merged, but I stored the mask
    # for code clarity.
    mask = (x[np.newaxis,:]-cx)**2 + (y[:,np.newaxis]-cy)**2 < r1**2
    mask2 = (x[np.newaxis,:]-cx)**2 + (y[:,np.newaxis]-cy)**2 < r2**2
    arr[mask2] = 1.
    arr[mask] = 0.
    
    # This plot shows that only within the circle the value is set to 123.
    plt.figure(figsize=(6, 6))
    plt.pcolormesh(x, y, arr)
    plt.colorbar()
    plt.show()
    
    
    return arr

def get_circlar_var(inner, outer, arr):
    
    mask_20th_8th = get_circluar_mask(len(arr[0]), len(arr[0])/2,len(arr[0])/2, len(arr[0])/inner,  len(arr[0])/outer)
    
    mask_arr = mask_20th_8th.flatten()
    val = arr.flatten()
    
    subset = []
    for i in range(0, len(val)):
        if mask_arr[i]:
            subset.append(val[i])
    
    return np.var(subset)

    
print (get_circlar_var(20, 8, R_LH))
print (get_circlar_var(8, 4, R_LH))

fig = plt.figure(figsize=(24, 6))
for i, a in enumerate([R_LL, R_LH, R_HL, R_HH]):
    ax = fig.add_subplot(1, 4, i + 1)
    ax.imshow(a)
    
    ax.set_title(titles[i], fontsize=10)
    ax.set_xticks([])
    ax.set_yticks([])
    
    
    
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(R_LH[80:120, 80:120])
plt.show()


In [ ]:
import random

#candidates for presentation:
#1)  midsize file = '/home/fun/data/AL162003/10_modis_satellite_20031005.npy'
#2) /home/fun/data/AL052001/10_modis_satellite_20010828.npy

# file ='/home/fun/data/AL182012/10_modis_satellite_20121023.npy'

# rgb = np.load(file)


# plt.imshow(rgb, interpolation='nearest', aspect='auto')
# #plt.imshow(T20)

# #plt.show()

# rgb.shape

# red = rgb[:, :, 0]
# green = rgb[:, :, 1]
# blue = rgb[:, :, 2]


def neighborRGB(color):
    for row in range(len(color)):

        found_swath = False
        swath_start = 0
        len_swath = 0
        idx = 0

        curr_row = color[row]
        
        while idx < len(curr_row):
            
            if found_swath == True:
                if curr_row[idx][0] != -2.8672 or curr_row[idx][1] != -2.8672 or curr_row[idx][2] != -2.8672:
                    found_swath = False
                else:
                    len_swath += 1
            elif curr_row[idx][0] == -2.8672  and curr_row[idx][1] == -2.8672 and curr_row[idx][2] == -2.8672:
                found_swath = True
                len_swath = 1
                swath_start = idx
            idx += 1

        existing_vals = []

        left_length = max(1,int(len_swath/2))
        right_length = max(1,int(len_swath - left_length))
        swath_end = swath_start + len_swath + 1
        #print("left:",left_length, "right:", right_length, "swath start:", swath_start)
        
        extra_right = max(0, swath_end+right_length - len(curr_row))
        left_length += extra_right
        
        extra_left = max(0, left_length - swath_start)
        right_length += extra_left

        existing_vals = np.append(existing_vals, curr_row[swath_start-left_length:swath_start])
        existing_vals = np.append(existing_vals, curr_row[swath_end:swath_end+right_length])
        
        
        for new_row_idx in (idx+1, idx+10):
            if new_row_idx >= 0 and new_row_idx < len(color):  # add previous row
                new_row = color[new_row_idx]
                for i in range(swath_start-left_length, swath_start):
                    if i >=0 and i < len(new_row):
                        if new_row[i][0] != -2.8672 or new_row[i][1] != -2.8672 or new_row[i][2] != -2.8672:
                            existing_vals = np.append(existing_vals, new_row[i])
                            
                for i in range(swath_end, swath_end+right_length+1):
                    if i >=0 and i < len(new_row):
                        if new_row[i][0] != -2.8672 or new_row[i][1] != -2.8672 or new_row[i][2] != -2.8672:
                            existing_vals = np.append(existing_vals, new_row[i])
        
    
      
    
        for idx in range(len_swath):
            rand_idx = random.randint(0, len(existing_vals)-1)
            curr_row[swath_start+idx] = existing_vals[rand_idx]
            

#neighborRGB(rgb)
    


# rgb_array = np.dstack([red, green, blue])


# rgb_array = ((rgb_array * 10000 + 100) / 66.66)*1.5


# rgb_array = rgb_array.astype(int)

# rgb_array[rgb_array < 0] = 0
# rgb_array[rgb_array > 255] = 255

# red = rgb_array[:, :, 0]
# green = rgb_array[:, :, 1]
# blue = rgb_array[:, :, 2]



# fig, ax = plt.subplots(figsize=(8, 8))
# ax.imshow(rgb_array, interpolation='nearest')
# plt.show()


# fig.tight_layout()
# plt.show()

# R_coeffs2 = pywt.dwt2(red, 'bior1.3')
# G_coeffs2 = pywt.dwt2(green, 'bior1.3')
# B_coeffs2 = pywt.dwt2(blue, 'bior1.3')

# titles = ['Approximation', ' Horizontal detail',
#           'Vertical detail', 'Diagonal detail']

# R_LL, (R_LH, R_HL, R_HH) = R_coeffs2
# G_LL, (G_LH, G_HL, G_HH) = G_coeffs2
# B_LL, (B_LH, B_HL, B_HH) = B_coeffs2

# def getsquare(start, end, arr):
#     print(np.var(arr[start:end, start:end].flatten()))

# print (R_LH[75:125, 75:125])
# getsquare(0, 50, R_LH)
# getsquare(50, 100, R_LH)
# getsquare(75, 125, R_LH)
# getsquare(60, 140, R_LH)

# fig = plt.figure(figsize=(24, 6))
# for i, a in enumerate([R_LL, R_LH, R_HL, R_HH]):
#     ax = fig.add_subplot(1, 4, i + 1)
#     ax.imshow(a, interpolation="nearest", cmap=plt.cm.gray)
    
#     ax.set_title(titles[i], fontsize=10)
#     ax.set_xticks([])
#     ax.set_yticks([])


In [ ]:
import tensorflow
import keras
from keras import layers

# This is the size of our encoded representations
encoding_dim = 40000  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# This is our input image
input_img = keras.Input(shape=(40000,))
# "encoded" is the encoded representation of the input
encoded = layers.Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = layers.Dense(200, activation='sigmoid')(encoded)

# This model maps an input to its reconstruction
autoencoder = keras.Model(input_img, decoded)

In [ ]:
encoder = keras.Model(input_img, encoded)

encoded_input = keras.Input(shape=(encoding_dim,))
# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# Create the decoder model
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

In [ ]:
red = red.astype("float32")

In [ ]:
autoencoder.compile(optimizer="adam"
                    , loss='binary_crossentropy')

autoencoder.fit(red, red,
                epochs=50,
                validation_data=(rgb, rgb))

In [ ]:
hdf = SD('/home/fun/data/AL182012/MOD09CMG.A2012298.006.2015250045544.hdf')

In [ ]:
def getTimeIndex(lon, lat):
    n = 3600
    m = 7200

    interval = 180.0 / n

    lat_index = int((90 - lat) / interval)
    long_index = int((lon + 180) / interval)
    
    return long_index, lat_index

def getTgtArea(global_map, lon, lat, radius):
    n = 3600
    m = 7200

    interval = 180.0 / n

    lat_index = int((90 - lat) / interval)
    long_index = int((lon + 180) / interval)

    n_radius = int(radius / interval)

    return global_map[lat_index - n_radius:lat_index + n_radius, long_index - n_radius:long_index + n_radius]

#def interpolateLocation(day, path, time_array):

In [ ]:
for it in hdf.datasets():
    print (it)

time = hdf.select('Coarse Resolution Granule Time').get()

R = hdf.select('Coarse Resolution Surface Reflectance Band 1').get()
B = hdf.select('Coarse Resolution Surface Reflectance Band 3').get()
G = hdf.select('Coarse Resolution Surface Reflectance Band 4').get()

R_true = R * 0.0001
G_true = G * 0.0001
B_true = B * 0.0001

rgb = np.dstack([R_true, G_true, B_true])

In [ ]:
name = '/home/fun/data/AL182012' + '/profile.json'
f = open(name)
profile = json.load(f)
paths = profile['path']

select_path = paths[9]

print(select_path)

lat = str(select_path['lt'])
lon = str(select_path['lg'])
lom = -1 if lon[-1] == 'W' else 1

lat = float(lat[:-1])
lon = float(lon[:-1])

print(lat, lon)

lon_index, lat_index, = getTimeIndex(lon, lat)

print(lon_index, lat_index)

In [ ]:
selected_time = (time[lon_index, lat_index])/60
selected_time # hours after 00:00, 12 AM

In [ ]:
time_array = getTgtArea(time, lon, lat, 12.5)
selected_rgb = getTgtArea(rgb, lon, lat, 12.5)
plt.imshow(time_array)
plt.show()
plt.imshow(selected_rgb)
plt.show()

# swath gaps come from missing satellite coverage, resulting in non-smooth time differences

print(selected_time)
time_array

In [ ]:
def getNearestIndex(array, value):
    idx = (np.abs(array - value)).argmin()
    return idx

def getScaleFactor(small_time, large_time, target_time): # coefficient of multiplication for lin interpolation
    delta_time = large_time - small_time
    delta_target = target_time - small_time
    scale_factor = delta_target/delta_time
    return scale_factor

def coordSTRtoFLOAT(coords): # called in linearInterpolation, shape (2,), lat first then lon
    lat = str(coords[0])
    lon = str(coords[1])
    lom = -1 if lon[-1] == 'W' else 1

    lat = float(lat[:-1])
    lon = float(lon[:-1])
    lon = lon * lom
    
    return lat, lon


def linearCalculation(scale_factor, small_coords, large_coords): # coords passed as shape (2,)

    sm_lat, sm_lon = coordSTRtoFLOAT(small_coords)
    lg_lat, lg_lon = coordSTRtoFLOAT(large_coords)
    
    delta_lat = lg_lat - sm_lat
    delta_lon = lg_lon - sm_lon
    
    scale_lat = delta_lat * scale_factor
    scale_lon = delta_lon * scale_factor
    
    interp_lat = scale_lat + sm_lat
    interp_lon = scale_lon + sm_lon
    
    return interp_lat, interp_lon

def wrappedLinearInterpolation(paths, day, selected_time):
    early_times = []
    early_coords = []
    late_times = []
    late_coords = []
    for path in paths:

        if path['date'] == int(day):
            path_time_hour = float(path['time'])/60
            if path_time_hour <= selected_time:
                early_times = np.append(early_times, path_time_hour)
                early_coords = np.append(early_coords, (path['lt'], path['lg']))
                early_coords = early_coords.reshape(int(len(early_coords)/2), 2)
            else:
                late_times = np.append(late_times, path_time_hour)
                late_coords = np.append(late_coords, (path['lt'], path['lg']))
                late_coords = late_coords.reshape(int(len(late_coords)/2), 2)

    early_index = getNearestIndex(early_times, selected_time)
    late_index = getNearestIndex(late_times, selected_time)

    scale_factor = getScaleFactor(early_times[early_index], late_times[late_index], selected_time)

    new_lon, new_lat = linearCalculation(scale_factor, early_coords[early_index], late_coords[late_index])

    return new_lon, new_lat

def getClosestTime(profile, time_array, day):
    paths = profile['path']
    paths_in_day = []
    time_diffs = []
    for path in paths:
        if path['date'] == int(day):
            paths_in_day = np.append(paths_in_day, path)
    
    for path in paths_in_day:
        # get lt, lg, pass through getTimeIndex(lon, lat)
        # get time[new_lat, new_lon], append to time_diffs
        # time_diffs = time_diffs/60
        # idx = getNearestIndex(time_diffs, early_time)
        # if time_diffs[idx] - early_time >= (late_time - early_time)/2:
            # ...
        # 
    getNearestIndex(array, value)
        

In [ ]:
paths_in_day = []
for path in paths:
    if path['date'] == int(20121024):
        day = np.append(day, path)
        
        
day[1]

In [ ]:
new_lon, new_lat = wrappedLinearInterpolation(paths, '20121024', selected_time)

print(new_lon, new_lat)

new_rgb = getTgtArea(rgb, new_lon, new_lat, 12.5)

plt.imshow(new_rgb)

In [ ]:
interp_indices = getTimeIndex(interpolated[1], interpolated[0])
print(interp_indices)
new_time = time[interp_indices[0], interp_indices[1]]
print(new_time/60)

In [ ]:
# HURRICANE IDA (AL092021) ON AUGUST 29TH (20210829) AFTER HCI
file ='/home/fun/data/AL092021/10_modis_satellite_20210829.npy'

rgb = np.load(file)

R_true = rgb[:, :, 0]
B_true = rgb[:, :, 2]
G_true = rgb[:, :, 3]

R_true = R_true * 0.00010
G_true = G_true * 0.00010
B_true = B_true * 0.00010

rgb_display = np.dstack([R_true, G_true, B_true])
    
fig = plt.figure(figsize=(6, 6))
fig.tight_layout()
plt.imshow(rgb_display, interpolation='nearest', aspect='auto')

# rgb = ((rgb * 10000 + 100) / 66.66)*1.5


# rgb = rgb.astype(int)

# rgb[rgb < 0] = 0
# rgb[rgb > 255] = 255





# plt.imshow(rgb, interpolation='nearest', aspect='auto')
# #plt.imshow(T20)

# #plt.show()




# red = rgb[:, :, 0]
# green = rgb[:, :, 1]
# blue = rgb[:, :, 2]


# print (green.max(), red.max(), blue.max())
# print (green.min(), red.min(), blue.min())




In [ ]:
# HURRICANE IDA (AL092021) ON AUGUST 29TH (20210829) AFTER HCI
file ='/home/fun/data/AL092021/10_modis_satellite_20210829.npy'

rgb = np.load(file)

R_true = rgb[:, :, 0]
B_true = rgb[:, :, 2]
G_true = rgb[:, :, 3]

R_true = R_true * 0.00010
G_true = G_true * 0.00010
B_true = B_true * 0.00010

rgb_display = np.dstack([R_true, G_true, B_true])
    
fig = plt.figure(figsize=(6, 6))
fig.tight_layout()
plt.imshow(rgb_display, interpolation='nearest', aspect='auto')

In [ ]:
# HURRICANE TEDDY ON SEPTEMBER 17TH 2020 FOR THE DSGF ALGO

file ='/home/fun/data/AL202020/10_modis_satellite_20200917.npy'

rgb = np.load(file)

R_true = rgb[:, :, 0]
B_true = rgb[:, :, 2]
G_true = rgb[:, :, 3]

R_true = R_true * 0.00010
G_true = G_true * 0.00010
B_true = B_true * 0.00010

rgb_display = np.dstack([R_true, G_true, B_true])
    
fig = plt.figure(figsize=(6, 6))
fig.tight_layout()
plt.imshow(rgb_display, interpolation='nearest', aspect='auto')

In [ ]:
# HURRICANE SAM ON OCTOBER 1ST 2021 FOR THE HCI ALGO
file ='/home/fun/data/AL182021/10_modis_satellite_20211001.npy'

rgb = np.load(file)

R_true = rgb[:, :, 0]
B_true = rgb[:, :, 2]
G_true = rgb[:, :, 3]

R_true = R_true * 0.00011
G_true = G_true * 0.00011
B_true = B_true * 0.00011

rgb_display = np.dstack([R_true, G_true, B_true])
    
fig = plt.figure(figsize=(6, 6))
fig.tight_layout()
plt.imshow(rgb_display, interpolation='nearest', aspect='auto')

In [ ]:
sst_test = np.load('/home/fun/data/AL092021/noaa_sst_20210829.npy')
sst_test = sst_test.reshape(200, 200)

print(sst_test)

fig, ax = plt.subplots()
shw = ax.imshow(sst_test, vmin = -10, vmax = 30)
bar = plt.colorbar(shw)
  
# show plot with labels
bar.set_label('Sea Skin Temperature (C)')
plt.show()

In [ ]:
imerg_test = np.load('/home/fun/data/AL092021/imerg_precipitation_20210829_0000.npy')
imerg_test = imerg_test.reshape(200, 200)
print(imerg_test)

fig, ax = plt.subplots()
shw = ax.imshow(imerg_test, vmin = 0, vmax = 20)
bar = plt.colorbar(shw)
  
# show plot with labels
bar.set_label('Amount of Precipitation (mm/hr)')
plt.show()

In [ ]:
import umap